In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import math


def guided_filter(I,p,win_size,eps):

    mean_I = cv2.blur(I,(win_size,win_size))
    mean_p = cv2.blur(p,(win_size,win_size))

    corr_I = cv2.blur(I*I,(win_size,win_size))
    corr_Ip = cv2.blur(I*p,(win_size,win_size))

    var_I = corr_I-mean_I*mean_I
    cov_Ip = corr_Ip - mean_I*mean_p

    a = cov_Ip/(var_I+eps)
    b = mean_p-a*mean_I

    mean_a = cv2.blur(a,(win_size,win_size))
    mean_b = cv2.blur(b,(win_size,win_size))

    q = mean_a*I + mean_b
    return q
def get_min_channel(img):
    return np.min(img,axis=2)
def get_max_channel(img):
    return np.max(img,axis=2)
def min_filter(img,r=7):
    side_len = 2*r+1
    kernel = np.ones((side_len,side_len))
    return cv2.erode(img,kernel)#最小值滤波器，可用腐蚀替代
def max_filter(img,r=7):
    side_len = 2*r+1
    kernel = np.ones((side_len,side_len))
    return cv2.dilate(img,kernel)#最大值滤波器，用膨胀替代

def get_A(img_haze,name,dark_channel,bins_l,rate=0.001):
    hist,bins = np.histogram(dark_channel,bins=bins_l)#得到直方图
    num = dark_channel.size * (1-rate)
    d = np.cumsum(hist)#累加
    threshold=0
    for i in range(bins_l-1,0,-1):
        if d[i]<=num:
            threshold=i
            break
    A = img_haze[dark_channel>=bins[threshold]].max()
    #候选区域标记为红色
    show  = np.copy(img_haze)
    show[dark_channel>=bins[threshold]] = 0,0,255
    cv2.imwrite('./result0/most_haze_opaque_region/most_haze_opaque_region'+name+'.jpg',show)
    return A
def get_t(dark_channel,A,t0=0.1,w=0.95):
    dark_channel = np.float32(dark_channel)
    t = 1-w*dark_channel/A #需要乘上一系数w，为远处的物体保留少量的雾
    t = np.clip(t,t0,1)#论文中提到t(x)趋于0容易产生噪声，所以设置一最小值0.1
    return t
def dark_channel_dehaze(I,name):
    dark_channel = get_min_channel(I)
    dark_channel_1 = min_filter(dark_channel,r=7)
    #cv2.imwrite('./result0/dark_channel/dark_channel'+name+'.png', dark_channel_1)

    A = get_A(I,name,dark_channel_1,bins_l=256)
    print(A)
    t = get_t(dark_channel_1,A)
    #cv2.imwrite('./result0/rough_t/rought'+name+'.png', t*255.0)
    
    
#     gray_I = cv2.imread('./HR0/'+str(0) + '.png',cv2.IMREAD_GRAYSCALE)
    gray_I = cv2.cvtColor(I,cv2.COLOR_BGR2GRAY)
    
    t = guided_filter(gray_I/255.0,t,15,0.001)
    #cv2.imwrite('./gray_optimalt'+name+'.png', t*255.0)
    
    #cv2.imshow('dark_t',t)
    t = t[:,:,np.newaxis].repeat(3,axis=2)#升维至(r,w,3)

    I = np.float32(I)#先变为float32再参与计算，否则结果很不一样
    J = (I-A)/t + A

    J = np.clip(J,0,255)#这两句位置不能交换，本身J里面数据的取值是可能超过255（uint8的取值上限）
    J = np.uint8(J)
    return J

In [2]:
i = 301
I = cv2.imread('./images/'+str(i) + '.png')
J = dark_channel_dehaze(I,str(i))


cv2.imshow('dark_J',J)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [3]:
def max_min_filter(image,r=7,flag=True,divide=1,borderType=cv2.BORDER_REPLICATE):
    img = image.copy()
    new_img = cv2.copyMakeBorder(img,r,r,r,r,borderType)
    h,w = img.shape
    for i in range(h//divide):
        for j in range(w):
            if flag:
                img[i,j]=new_img[i:i+2*r+1,j:j+2*r+1].max()
            else:
                img[i,j]=new_img[i:i+2*r+1,j:j+2*r+1].min()
    return img
    

#对亮度限制对比度自适应直方图均衡化CLAHE
def clahe_v(image,clipLimit=2.0, tileGridSize=(8, 8)):
    img_hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(img_hsv)
    clahe = cv2.createCLAHE(clipLimit, tileGridSize)
    v = clahe.apply(v)
    image_clahe = cv2.merge([h, s, v])
    image_clahe = cv2.cvtColor(image_clahe, cv2.COLOR_HSV2BGR)
    return image_clahe
# 限制对比度自适应直方图均衡化CLAHE
def clahe(image,clipLimit=2.0, tileGridSize=(8, 8)):
    b, g, r = cv2.split(image)
    clahe = cv2.createCLAHE(clipLimit, tileGridSize)
    b = clahe.apply(b)
    g = clahe.apply(g)
    r = clahe.apply(r)
    image_clahe = cv2.merge([b, g, r])
    return image_clahe

In [107]:
img = clahe_v(I_int,2)
cv2.imshow('img',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [4]:
import time
def dark_channel_dehaze(I,name):
    dark_channel = get_min_channel(I)
    dark_channel_1 = max_min_filter(dark_channel,r=7,flag=False)
    #dark_channel_1 = min_filter(dark_channel,r=7)
    #cv2.imwrite('./result0/dark_channel/dark_channel'+name+'.png', dark_channel_1)

    #A = get_A(I,name,dark_channel_1,bins_l=256)
    #print(A)
    A=255
    t = get_t(dark_channel_1,A)
    #cv2.imwrite('./result0/rough_t/rought'+name+'.png', t*255.0)
    

    gray_I = cv2.cvtColor(I,cv2.COLOR_BGR2GRAY)
    
    t = guided_filter(gray_I/255.0,t,15,0.001)
    #cv2.imwrite('./result0/gray_optimalt'+name+'.png', t*255.0)
    
    #cv2.imshow('dark_t',t)
    t = t[:,:,np.newaxis].repeat(3,axis=2)#升维至(r,w,3)

    I = np.float32(I)#先变为float32再参与计算，否则结果很不一样
    J = (I-A)/t + A

    J = np.clip(J,0,255)#这两句位置不能交换，本身J里面数据的取值是可能超过255（uint8的取值上限）
    J = np.uint8(J)
    return J

i = 110
I = cv2.imread('./images/'+str(i) + '.png')
time1 = time.time()
J = dark_channel_dehaze(I,str(i))
time2 = time.time()
print("time:"+str(time2-time1))

cv2.imshow('dark_J',J)
cv2.imwrite('./result_improved/dark_J_' + str(i) +'.png',J)
cv2.waitKey(0)
cv2.destroyAllWindows()

time:2.7626140117645264
